In [4]:
import torch
import data

In [106]:
words = 10
temperature = 0.1

In [79]:
device = torch.device("cuda")
with open("./model.pt", 'rb') as f:
    model = torch.load(f, map_location=device)
model.eval()

/tmp/ipykernel_43623/3239447825.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(f, map_location=device)


TransformerModel(
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=200, out_features=200, bias=True)
        )
        (linear1): Linear(in_features=200, out_features=200, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=200, out_features=200, bias=True)
        (norm1): LayerNorm((200,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((200,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
    (norm): LayerNorm((200,), eps=1e-05, elementwise_affine=True)
  )
  (decoder): Linear(in_features=200, out_features=33278, bias=True)
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (input_emb): Embedding(33278, 200)
)

In [80]:
def tokenize_line(line) -> list:
    dictionary = data.Dictionary()
    idss = []
    words = line.split() + ["<eos>"]
    for word in words:
        dictionary.add_word(word)
    for word in words:
        idss.append(dictionary.word2idx[word])
    idss.append(torch.tensor(idss).type(torch.int64))
    return idss, dictionary

In [104]:
corpus = data.Corpus('./data/wikitext-2')

input_str = "This is a test, it is"
tokens, t_dict = tokenize_line(input_str)
print(tokens)
print(t_dict.idx2word)
n_tokens = len(t_dict)
print("n_tokens", n_tokens)
model_input = torch.randint(len(corpus.dictionary), (1, 1), dtype=torch.long).to(device)
print(model_input)

[0, 1, 2, 3, 4, 1, 5, tensor([0, 1, 2, 3, 4, 1, 5])]
['This', 'is', 'a', 'test,', 'it', '<eos>']
n_tokens 6
tensor([[5670]], device='cuda:0')


In [114]:
output_text = ""
with torch.no_grad():
    for i in range(words):
        output = model(model_input)
        word_weights = output[-1].squeeze().div(0.5).exp().cpu()
        word_idx = torch.multinomial(word_weights, 1)[0]
        word_tensor = torch.Tensor([[word_idx]]).long().to(device)
        model_input = torch.cat([model_input, word_tensor], 0)     
        word = corpus.dictionary.idx2word[word_idx]
        output_text += (" " + word)
print(output_text)

 kakapo is a female , the female . <eos> The
